### TPC Benchmarking  
Results Plotting  

In [1]:
import os
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

sns.set_context("notebook", font_scale=1.0, rc={"lines.linewidth": 1.0})

pd.options.display.max_rows = 999
pd.options.display.max_columns = 999

import config, sfa, bqa, tools

In [2]:
from importlib import reload

In [3]:
results_dir = "/home/colin/code/bq_snowflake_benchmark/results/bqsf_ds_01_results-ds_1GB_01-dev_test_04-2020-05-21_19:04:08.344863/"

In [4]:
def exp_log_t0(results_dir):
    """Get query history for experiment"""
    fps_time = glob.glob(results_dir + "/*_*_times*")
    source_csv = {k:v for k,v in zip([os.path.basename(x)[:2] for x in fps_time], fps_time)}
    dfsf = pd.read_csv(source_csv["sf"])
    dfbq = pd.read_csv(source_csv["bq"])
    df = pd.concat([dfsf, dfbq])
    t0 = df.driver_t0.min()
    return t0

In [5]:
t0 = exp_log_t0(results_dir)
t0

'2020-05-22 02:04:12.238239+00:00'

In [6]:
t_buffer = pd.Timedelta("20 minutes")
t_buffer

Timedelta('0 days 00:20:00')

In [7]:
reload(sfa)
sf = sfa.SFTPC(test="ds",
               scale=1,
               cid="01",
               warehouse="TEST9000",
               desc="query_history",
               verbose=False,
               verbose_query=False)
sf.connect()
df_sf_history_sq, qid_sf = sf.query_history(t0=pd.to_datetime(t0) - t_buffer,
                                            t1=pd.Timestamp.now())

sf.close()

In [8]:
%store df_sf_history_sq

Stored 'df_sf_history_sq' (DataFrame)


In [9]:
reload(sfa)
sf = sfa.AU(warehouse="TEST9000")
sf.connect()

df_sf_history_av, qid_sf = sf.query_history_view(t0=pd.to_datetime(t0) - t_buffer,
                                                 t1=pd.Timestamp.now())

sf.close()

In [10]:
%store df_sf_history_av

Stored 'df_sf_history_av' (DataFrame)


In [11]:
df_sf_history_av.to_csv(results_dir + config.sep + "sf_query_history.csv")

In [12]:
reload(bqa)
bq = bqa.BQTPC(test="ds",
               scale=1,
               cid="01",
               desc="query_history",
               verbose=False,
               verbose_query=True)

df_bq_history, qid_bq = bq.query_history(t0=pd.to_datetime(t0) - t_buffer,
                                         t1=pd.Timestamp.now())

QUERY TEXT
select * from `region-us`.INFORMATION_SCHEMA.JOBS_BY_PROJECT where job_type = 'QUERY' and end_time between '2020-05-22 01:44:12' AND '2020-05-21 19:20:05'



In [13]:
%store df_bq_history

Stored 'df_bq_history' (DataFrame)


In [14]:
df_bq_history.to_csv(results_dir + config.sep + "bq_query_history.csv")